In [1]:
import numpy as np
import cv2

In [2]:
def erod(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross' or type_of_mask == 'vertline' or type_of_mask == 'horizline'
    result = image.copy()
    if type_of_mask == 'square':
        result = erod_square(result, size)
    elif type_of_mask == 'cross':
        result = erod_cross(result, size)
    elif type_of_mask == 'vertline':
        result = erod_vert_line(result, size)
    elif type_of_mask == 'horizline':
        result = erod_horiz_line(result, size)
    return result

In [3]:
def process_operation(image, i, j, mask, operation):
    offset = int(mask.shape[0] / 2)
    mask_i = 0
    mask_j = 0
    right_border = offset
    if mask.shape[0] % 2 != 0:
        right_border += 1
    if operation == 'erod':
        for k in range(i - offset, i + right_border):
            mask_j = 0
            for l in range(j - offset, j + right_border):
                if mask[mask_i, mask_j] == 1 and image[k, l] != 0:
                    return 255
                mask_j += 1
            mask_i += 1
    elif operation == 'dilat':
        for k in range(i - offset, i + right_border):
            mask_j = 0
            for l in range(j - offset, j + right_border):
                if mask[mask_i, mask_j] == 1 and image[k, l] == 0:
                    return 0
                mask_j += 1
            mask_i += 1
        return 255
    return 0
    

In [4]:
def erod_square(image, size):
    result = image.copy()
    status = True
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, np.ones((size, size)), 'erod')
    return result

In [5]:
def erod_cross(image, size):
    result = image.copy()
    status = True
    mask = create_cross(size)
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result

In [6]:
def create_cross(size):
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[center_point, i] = 1
        mask[i, center_point] = 1
        
    return mask

In [7]:
def dilat(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    if type_of_mask == 'square':
        result = dilat_square(result, size)
    elif type_of_mask == 'cross':
        result = dilat_cross(result, size)
    return result

In [8]:
def dilat_square(image, size):
    result = image.copy()
    status = False
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, np.ones((size, size)), 'dilat')
    return result

In [9]:
def dilat_cross(image, size):
    result = image.copy()
    status = False
    mask = create_cross(size)
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'dilat')
    return result

In [10]:
def opening(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = erod(result, size, type_of_mask)
    result = dilat(result, size, type_of_mask)
    return result

In [11]:
def closing(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = dilat(result, size, type_of_mask)
    result = erod(result, size, type_of_mask)
    return result

In [12]:
def filter(image):
    result = image.copy()
    for i in range(3, image.shape[0] - 3):
        for j in range(3, image.shape[1] - 3):
            if image[i, j] == 0:
                if image[i, j + 1] == 255 and image[i, j - 1] == 255 and image[i + 1, j] == 255 and image[i - 1, j] == 255:
                    result[i, j] = 255
            elif image[i, j] == 255:
                if image[i, j + 1] == 0 and image[i, j - 1] == 0 and image[i + 1, j] == 0 and image[i - 1, j] == 0:
                    result[i, j] = 0
    return result

In [13]:
def make_noise(image, prob):
    result = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            p = np.random.randint(0, 101)
            if p < prob :
                p_color = np.random.randint(0, 101)
                if p_color < 50:
                    result[i, j] = 0
                else:
                    result[i, j] = 255
    return result

In [14]:
def circle(image, centre, radius):
    circle = [centre, radius]
    for i in range(circle[0][0] - circle[1], circle[0][0] + circle[1]):
        for j in range(circle[0][1] - circle[1], circle[0][1] + circle[1]):
            if (i - circle[0][0]) ** 2 + (j - circle[0][1]) ** 2 < circle[1] ** 2:
                image[i, j] = 0

In [15]:
def triangle(image, A, B, C):
    triangle = [A, B, C]
    x_a = triangle[0][1]
    y_a = triangle[0][0]
    x_b = triangle[1][1]
    y_b = triangle[1][0]
    x_c = triangle[2][1]
    y_c = triangle[2][0]

    left = triangle[0][0]
    right = triangle[0][0]
    above = triangle[0][1]
    below = triangle[0][1]
    for i in triangle:
        if left > i[0]:
            left = i[0]
        if right < i[0]:
            right = i[0]
        if above > i[1]:
            above = i[1]
        if below < i[1]:
            below = i[1]
    for i in range(above, below):
        for j in range(left, right):
            coord_1 = (x_a - i) * (y_b - y_a) - (x_b - x_a) * (y_a - j)
            coord_2 = (x_b - i) * (y_c - y_b) - (x_c - x_b) * (y_b - j)
            coord_3 = (x_c - i) * (y_a - y_c) - (x_a - x_c) * (y_c - j)
            if coord_1 >= 0 and coord_2 >= 0 and coord_3 >= 0:
                image[i, j] = 0

In [16]:
def square(image, A, B):
    square = [A, B]
    for i in range(square[0][0], square[1][0]):
        for j in range(square[0][1], square[1][1]):
            image[i, j] = 0

In [17]:
def noise_reduce(image, filter_image, noised_image):
    assert image.shape == filter_image.shape and image.shape == noised_image.shape
    count_noised = 0
    count_filtered = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] - noised_image[i, j] != 0:
                count_noised += 1
            if image[i, j] - filter_image[i, j] != 0:
                count_filtered += 1
    return count_noised / count_filtered

# 1. Фильтрация бинарных изображений

## 1.1. Создать или выбрать бинарное изображение с любым простым рисунком, состоящим из примитивных фигур (круги, квадраты, полигоны).

In [18]:
image = np.zeros((500, 500))
image.fill(255)

In [19]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [20]:
path = 'laba2/'
cv2.imwrite(path + 'image.png', image)

True

## 1.2. На полученное изображение наложить шум (1%, 2%, 5%, 10%, 20%, 50%).

In [48]:
noised = []
for i in [1, 2, 5, 10, 20, 50]:
    noise = make_noise(image, i)
    noised.append(noise)
    cv2.imwrite(path + 'noise/image_noise_' + str(i) + '%' + '.png', noise)

## 1.3. Выполнить операции эрозии и дилатации для различных структурных элементов (крест, квадрат). Визуально оценить результаты эрозии и дилатации.

In [31]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'square')
    cv2.imwrite(path + 'erod/erod_image_square_size' + str(i) + '.png', erod_image)

In [32]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'cross')
    cv2.imwrite(path + 'erod/erod_image_cross_size'+ str(i) + '.png', erod_image)

In [33]:
for i in range(3, 6):
    dilat_image = dilat(image, i, type_of_mask = 'square')
    cv2.imwrite(path + 'dilat/dilat_image_square_size' + str(i) + '.png', dilat_image)

In [34]:
for i in range(3, 6):
    dilat_image= dilat(image, i, type_of_mask = 'cross')
    cv2.imwrite(path + 'dilat/dilat_image_cross_size' + str(i) + '.png', dilat_image)

## 1.4. Провести фильтрацию шумов на изображении при помощи морфологических операций вскрытия и закрытия с различными размерами (от 3 до 5) и формами структурного элемента (крест, квадрат), а также алгоритма фильтрации шумов, которому соответствует логическая таблица истинности, представленная в табл. 1 со крестообразным структурным элементом размером 3.

In [21]:
noise_index = [1, 2, 5, 10, 20, 50]

In [25]:
opened_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'square')
        opened_image_square[i - 3, j] = open_image
        cv2.imwrite(path + 'open/open_image_square_noise' + str(noise_index[j]) + '%_size' + str(i) + '.png', open_image)

In [26]:
closed_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'square')
        closed_image_square[i - 3, j] = close_image
        cv2.imwrite(path + 'close/close_image_square' + str(noise_index[j]) + '%_size' + str(i) + '.png', close_image)

In [27]:
opened_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'cross')
        opened_image_cross[i - 3, j] = open_image
        cv2.imwrite(path + 'open/open_image_cross' + str(noise_index[j]) + '%_size' + str(i) + '.png', open_image)

In [28]:
closed_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'cross')
        closed_image_cross[i - 3, j] = close_image
        cv2.imwrite(path + 'close/close_image_cross' + str(noise_index[j]) + '%_size' + str(i) + '.png', close_image)

In [35]:
filtered_image = np.zeros((len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(len(noised)):
    filter_image = filter(noised[i])
    filtered_image[i] = filter_image
    cv2.imwrite(path + 'filter_image_' + str(i) + '.png', filter_image)

## 1.5. Рассчитать коэффициент снижения шума для различных методов фильтрации, для различной интенсивности шума и различных размеров и форм структурного элемента (окна).

In [30]:
for i in range(opened_image_square.shape[0]):
    for j in range(opened_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 7.68125
Size 3 Noised 1 reduce 8.243243243243244
Size 3 Noised 2 reduce 8.566433566433567
Size 3 Noised 3 reduce 7.376506024096385
Size 3 Noised 4 reduce 6.5333509793541555
Size 3 Noised 5 reduce 5.936137372178275
Size 4 Noised 0 reduce 1.2896117523609654
Size 4 Noised 1 reduce 2.0926243567753002
Size 4 Noised 2 reduce 3.656716417910448
Size 4 Noised 3 reduce 3.799255352156376
Size 4 Noised 4 reduce 3.533715103793844
Size 4 Noised 5 reduce 4.264310464310464
Size 5 Noised 0 reduce 4.083056478405315
Size 5 Noised 1 reduce 3.3936022253129345
Size 5 Noised 2 reduce 3.0655655655655654
Size 5 Noised 3 reduce 2.467257707032037
Size 5 Noised 4 reduce 2.463373253493014
Size 5 Noised 5 reduce 3.5884068112899463


In [31]:
for i in range(closed_image_square.shape[0]):
    for j in range(closed_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 1.0165425971877584
Size 3 Noised 1 reduce 0.9791332263242376
Size 3 Noised 2 reduce 0.872382851445663
Size 3 Noised 3 reduce 0.7302600190839694
Size 3 Noised 4 reduce 0.5403577136101929
Size 3 Noised 5 reduce 0.39593856369801755
Size 4 Noised 0 reduce 0.6075135936727633
Size 4 Noised 1 reduce 0.710334788937409
Size 4 Noised 2 reduce 0.6474630021141649
Size 4 Noised 3 reduce 0.4673129031026982
Size 4 Noised 4 reduce 0.3032272330806747
Size 4 Noised 5 reduce 0.2952403800019192
Size 5 Noised 0 reduce 0.9233658903080391
Size 5 Noised 1 reduce 0.7598878853939582
Size 5 Noised 2 reduce 0.4804298376343243
Size 5 Noised 3 reduce 0.28838907206782854
Size 5 Noised 4 reduce 0.19361645382950018
Size 5 Noised 5 reduce 0.27381423848279485


In [32]:
for i in range(opened_image_cross.shape[0]):
    for j in range(opened_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 9.67716535433071
Size 3 Noised 1 reduce 10.608695652173912
Size 3 Noised 2 reduce 11.600378787878787
Size 3 Noised 3 reduce 10.537865748709121
Size 3 Noised 4 reduce 10.327615062761506
Size 3 Noised 5 reduce 7.917395779722079
Size 4 Noised 0 reduce 1.6905089408528198
Size 4 Noised 1 reduce 3.080808080808081
Size 4 Noised 2 reduce 6.340579710144928
Size 4 Noised 3 reduce 8.574929971988796
Size 4 Noised 4 reduce 9.152020763811642
Size 4 Noised 5 reduce 7.173467008626719
Size 5 Noised 0 reduce 6.865921787709497
Size 5 Noised 1 reduce 6.759002770083103
Size 5 Noised 2 reduce 7.08912037037037
Size 5 Noised 3 reduce 7.001143510577473
Size 5 Noised 4 reduce 7.002269503546099
Size 5 Noised 5 reduce 5.6917953935806125


In [33]:
for i in range(closed_image_cross.shape[0]):
    for j in range(closed_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 1.0522260273972603
Size 3 Noised 1 reduce 1.0517241379310345
Size 3 Noised 2 reduce 1.0337552742616034
Size 3 Noised 3 reduce 1.005089058524173
Size 3 Noised 4 reduce 0.8961949023309854
Size 3 Noised 5 reduce 0.6473651541771432
Size 4 Noised 0 reduce 1.7658045977011494
Size 4 Noised 1 reduce 3.100381194409149
Size 4 Noised 2 reduce 4.57089552238806
Size 4 Noised 3 reduce 3.3910274162281917
Size 4 Noised 4 reduce 1.5294008302868827
Size 4 Noised 5 reduce 0.5711925294024821
Size 5 Noised 0 reduce 1.056749785038693
Size 5 Noised 1 reduce 1.0604085180356366
Size 5 Noised 2 reduce 1.0299310576761391
Size 5 Noised 3 reduce 0.9933479354263
Size 5 Noised 4 reduce 0.8019428831346047
Size 5 Noised 5 reduce 0.45667678469382456


In [52]:
for i in range(filtered_image.shape[0]):
        print('Noised {} reduce {}'.format(i, noise_reduce(image, filtered_image[i], noised[i])))

Noised 0 reduce 20.475409836065573
Noised 1 reduce 16.065359477124183
Noised 2 reduce 8.707801418439717
Noised 3 reduce 4.931709265175719
Noised 4 reduce 2.775897088693297
Noised 5 reduce 1.4120362889263371


# 2. Выделение контуров на бинарных изображениях

## 2.1. Создать или выбрать бинарное изображение, содержащее простой рисунок, состоящий из примитивных фигур (полосы, круги, квадраты).

In [53]:
image = np.zeros((500, 500))
image.fill(255)

In [54]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [55]:
cv2.imwrite(path + 'image.png', image)

True

## 2.2. С помощью морфологических операций выделить контур объекта. Выяснить, когда контур получается внешним, внутренним, четырёхсвязным, восьмисвязным.

In [60]:
def border(image, mask_type):
    assert mask_type == 'square' or mask_type == 'cross' or mask_type == 'vertline' or mask_type == 'horizline'
    
    result = image.copy()
    erod_image = erod(image, 3, mask_type)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if image[i, j] - erod_image[i, j] != 0:
                result[i, j] = 0
            else:
                result[i, j] = 255
    return result

In [61]:
b = border(image, 'horizline')
cv2.imwrite(path + 'border_horiz.png', b)

True

In [62]:
def erod_vert_line(image, size):
    result = image.copy()
    status = True
    
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[i, center_point] = 1

    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result

In [63]:
b = border(image, 'vertline')
cv2.imwrite(path + 'border_vert.png', b)

True

In [64]:
def erod_horiz_line(image, size):
    result = image.copy()
    status = True
    
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[center_point, i] = 1

    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result